Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE:
MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

MindSPONGE advanced tutorial 02: Bias potential and metadynamics (MetaD)

In [1]:
from mindspore import context
from mindsponge import Sponge
from mindsponge import ForceField
from mindsponge import set_global_units
from mindsponge import Protein
from mindsponge import WithEnergyCell
from mindsponge import UpdaterMD
from mindsponge.optimizer import SteepestDescent
from mindsponge.control import VelocityVerlet
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.control import Langevin
from mindsponge.sampling import Metadynamics
from mindsponge.function import VelocityGenerator
from mindsponge.colvar import Torsion
from mindsponge.function import PI

[WARNING] ME(185000:140541717730560,MainProcess):2023-03-24-17:39:25.936.302 [mindspore/run_check/_check_version.py:75] Can not found cuda libs, please confirm that the correct cuda version has been installed, you can refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")
set_global_units('nm', 'kj/mol')

In [3]:
system = Protein(pdb='alad.pdb')
potential = ForceField(system, 'AMBER.FF14SB')

In [4]:
phi = Torsion([4, 6, 8, 14])
psi = Torsion([6, 8, 14, 16])

In [5]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)
mini = Sponge(system, potential, min_opt, metrics={'phi': phi, 'psi': psi})

[MindSPONGE] Using fully connected neighbour list (not updated).


In [6]:
run_info = RunInfo(10)
mini.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-03-24 17:39:40
[MindSPONGE] Step: 0, E_pot: -55.75643, phi: -3.140886, psi: -3.1410766
[MindSPONGE] Step: 10, E_pot: -58.41481, phi: -3.1339903, psi: -3.1368215
[MindSPONGE] Step: 20, E_pot: -60.197777, phi: -3.1276336, psi: -3.1337032
[MindSPONGE] Step: 30, E_pot: -61.60443, phi: -3.1219037, psi: -3.1313035
[MindSPONGE] Step: 40, E_pot: -62.77914, phi: -3.1167603, psi: -3.1294105
[MindSPONGE] Step: 50, E_pot: -63.786407, phi: -3.1121268, psi: -3.1279025
[MindSPONGE] Step: 60, E_pot: -64.66443, phi: -3.107924, psi: -3.126701
[MindSPONGE] Step: 70, E_pot: -65.43921, phi: -3.1040812, psi: -3.125749
[MindSPONGE] Step: 80, E_pot: -66.13011, phi: -3.100538, psi: -3.125002
[MindSPONGE] Step: 90, E_pot: -66.75129, phi: -3.0972445, psi: -3.1244242
[MindSPONGE] Finished simulation at 2023-03-24 17:39:44
[MindSPONGE] Simulation time: 3.98 seconds.
--------------------------------------------------------------------------------


In [7]:
metad = Metadynamics(
    colvar=[phi, psi],
    update_pace=10,
    height=2.5,
    sigma=0.05,
    grid_min=-PI,
    grid_max=PI,
    grid_bin=360,
    temperature=300,
    bias_factor=100,
)

In [8]:
sim = WithEnergyCell(system, potential, bias=metad)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [9]:
vgen = VelocityGenerator(300)
velocity = vgen(system.coordinate.shape, system.atom_mass)
opt = UpdaterMD(
    system,
    integrator=VelocityVerlet(system),
    thermostat=Langevin(system, 300),
    time_step=1e-3,
    velocity=velocity
)

In [10]:
md = Sponge(sim, optimizer=opt, metrics={'phi': phi, 'psi': psi})

In [11]:
cb_h5md = WriteH5MD(system, 'tutorial_a02.h5md', save_freq=10)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-03-24 17:40:02
[MindSPONGE] Step: 0, E_pot: -67.31398, E_kin: 110.406906, E_tot: 43.092926, Temperature: 421.55234, phi: -3.1031148, psi: -3.113236
[MindSPONGE] Step: 10, E_pot: -21.638275, E_kin: 59.025436, E_tot: 37.38716, Temperature: 225.36916, phi: -3.138378, psi: -3.0506594
[MindSPONGE] Step: 20, E_pot: -13.997249, E_kin: 58.894936, E_tot: 44.897686, Temperature: 224.87088, phi: -3.0901785, psi: -3.0282526
[MindSPONGE] Step: 30, E_pot: -15.513279, E_kin: 68.00424, E_tot: 52.490963, Temperature: 259.65176, phi: -3.0149684, psi: -2.9992619
[MindSPONGE] Step: 40, E_pot: -13.233526, E_kin: 78.49111, E_tot: 65.25758, Temperature: 299.6924, phi: -2.9764829, psi: -2.9917028
[MindSPONGE] Step: 50, E_pot: -14.61941, E_kin: 83.28073, E_tot: 68.66132, Temperature: 317.97998, phi: -2.9594758, psi: -2.912541
[MindSPONGE] Step: 60, E_pot: -17.372175, E_kin: 77.772964, E_tot: 60.400787, Temperature: 296.9504, phi: -2.8799992, psi: -2.801089
[MindSPONGE] S